## Data Collection - Players

---

In [1]:
import pandas as pd
from functools import reduce

from client.models.player import Player
from client.scrapers.fifa import get_player_by_name

from client.scrapers.whoscored.player import crawl_player_overall_stats, crawl_player_defensive_stats, \
crawl_player_passing_stats, crawl_player_offensive_stats

In [2]:
league_team_df = pd.read_csv('./league/Premier League.csv')

In [3]:
players = []
for idx, row in league_team_df.iterrows():
    # excute player data crawling functions 
    overall_df = crawl_player_overall_stats(int(row.team_id))
    defensive_df = crawl_player_defensive_stats(int(row.team_id))
    passing_df = crawl_player_passing_stats(int(row.team_id))
    offensive_df = crawl_player_offensive_stats(int(row.team_id))

    # merge dataframes
    players_df = reduce(lambda x,y: pd.merge(x,y, on='name', how='outer'), [overall_df, defensive_df, passing_df, offensive_df])

    for i, player_row in players_df.iterrows():
        player: Player = get_player_by_name(player_name=player_row['name'], team_name=row.team_name, team_id=int(row.team_id))
        stats = player_row.to_dict()
        stats.pop('name')
        player.set_stats(data=player_row.to_dict())
        players.append(player)

In [8]:
len(players)

477

In [12]:
df = pd.DataFrame([p.__dict__ for p in players])

In [14]:
df.to_csv('./data/premier_league_players.csv', index=False)